In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 두 개의 데이터 불러오기
act = pd.read_csv('/Users/jisoyun/Desktop/sparta_python/Healthcare/sparta_15/data/dailyActivity_merged_fin_sum.csv')

In [2]:
act['Id'] = act['Id'].astype(str)
act['ActivityDate'] = pd.to_datetime(act['ActivityDate'])
 # 고유 사용자 수 확인

In [3]:
act['Id'].nunique()


35

In [4]:
act['ActivityDate'].nunique()

62

In [5]:
numeric_cols = act.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('Id') if 'Id' in numeric_cols else None

act = act.groupby(['Id', 'ActivityDate']).agg({
    **{col: 'sum' for col in numeric_cols}
}).reset_index()

In [6]:
MILE_TO_KM = 1.60934
distance_cols = [
    "TotalDistance","TrackerDistance","LoggedActivitiesDistance",
    "VeryActiveDistance","ModeratelyActiveDistance","LightActiveDistance"
]

for col in distance_cols:
    act[f"{col}_km"] = act[col] * MILE_TO_KM

# sedentaryActiveDistance는 분석에서 제외
act = act.drop(columns=["SedentaryActiveDistance"])

In [7]:
non_wear = (act['TotalSteps'] == 0) & (act['TotalDistance_km'] == 0) & (act['SedentaryMinutes'] >= 1380)
print(f"미착용일 수: {non_wear.sum()}개")
act = act[~non_wear].copy()

# 파생 변수 생성
act['weekday'] = act['ActivityDate'].dt.day_name()
act['is_weekend'] = act['weekday'].isin(['Saturday', 'Sunday'])
act['TotalActiveMinutes'] = (
    act['VeryActiveMinutes'] + 
    act['FairlyActiveMinutes'] + 
    act['LightlyActiveMinutes']
)

미착용일 수: 124개


In [8]:
#최소활동 필터링 
# 최소한의 활동이 있는 데이터만 사용
act_filtered = act[
    (act['TotalSteps'] > 1000) & 
    (act['TotalDistance_km'] > 0.5) &
    (act['TotalActiveMinutes'] > 30)
]
print(f"최소 활동 필터링 : {len(act_filtered)}개")
print(f"걸러진 행: {len(act) - len(act_filtered)}개")



최소 활동 필터링 : 1173개
걸러진 행: 76개


In [9]:
distance_columns = [col for col in act.columns if 'Distance' in col]
for col in distance_columns:
    act[col] = act[col].round(3)

act_filtered.shape

(1173, 23)

In [10]:
bins = [1000, 1500, 2000, 2500, float('inf')]
labels = ['1000-1500', '1500-2000', '2000-2500', '2500 이상']
act_filtered['CalorieGroup'] = pd.cut(act_filtered['Calories'], bins=bins, labels=labels, right=False)

print("✓ CalorieGroup 컬럼 추가 완료")
print(f"\n칼로리 그룹별 데이터 개수:")
print(act_filtered['CalorieGroup'].value_counts().sort_index())

✓ CalorieGroup 컬럼 추가 완료

칼로리 그룹별 데이터 개수:
CalorieGroup
1000-1500     86
1500-2000    307
2000-2500    314
2500 이상      463
Name: count, dtype: int64


In [11]:
# act를 actcp로 복사
actcp = act_filtered.copy()
print(f"✓ actcp 생성 완료 (행 개수: {len(actcp)})")

✓ actcp 생성 완료 (행 개수: 1173)


In [16]:
# 0값 확인
print("=== 0값 확인 ===")
numeric_cols = ['TotalSteps', 'TotalDistance_km', 'TrackerDistance_km',
                'LoggedActivitiesDistance_km', 'VeryActiveDistance_km',
                'ModeratelyActiveDistance_km', 'LightActiveDistance_km',
                'VeryActiveMinutes','FairlyActiveMinutes', 
                'LightlyActiveMinutes', 
                'SedentaryMinutes', 'Calories']

for col in numeric_cols:
    zero_count = (actcp[col] == 0).sum()
    if zero_count > 0:
        print(f"{col}: {zero_count}개 ({zero_count/len(actcp)*100:.1f}%)")

=== 0값 확인 ===
TrackerDistance_km: 3개 (0.3%)
LoggedActivitiesDistance_km: 1120개 (95.5%)
VeryActiveDistance_km: 444개 (37.9%)
ModeratelyActiveDistance_km: 399개 (34.0%)
LightActiveDistance_km: 1개 (0.1%)
VeryActiveMinutes: 439개 (37.4%)
FairlyActiveMinutes: 397개 (33.8%)
LightlyActiveMinutes: 1개 (0.1%)


In [12]:
# 3-3. 이상치 확인 (IQR 방법)
print("\n=== IQR 방법 이상치 개수 ===")

outlier_cols = ['TotalSteps', 'TotalDistance_km', 'TrackerDistance_km',
                'LoggedActivitiesDistance_km', 'VeryActiveDistance_km',
                'ModeratelyActiveDistance_km', 'LightActiveDistance_km',
                'VeryActiveMinutes','FairlyActiveMinutes', 
                'LightlyActiveMinutes','SedentaryMinutes', 'Calories']

for col in outlier_cols:
    Q1 = actcp[col].quantile(0.25)
    Q3 = actcp[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = actcp[(actcp[col] < lower_bound) | (actcp[col] > upper_bound)]
    
    if len(outliers) > 0:
        print(f"{col}: {len(outliers)}개 (하한: {lower_bound:.2f}, 상한: {upper_bound:.2f})")


=== IQR 방법 이상치 개수 ===
TotalSteps: 22개 (하한: -4227.50, 상한: 20456.50)
TotalDistance_km: 34개 (하한: -5.38, 상한: 23.91)
TrackerDistance_km: 34개 (하한: -5.42, 상한: 23.87)
LoggedActivitiesDistance_km: 53개 (하한: 0.00, 상한: 0.00)
VeryActiveDistance_km: 87개 (하한: -5.48, 상한: 9.13)
ModeratelyActiveDistance_km: 63개 (하한: -2.15, 상한: 3.58)
LightActiveDistance_km: 12개 (하한: -2.09, 상한: 14.00)
VeryActiveMinutes: 78개 (하한: -52.50, 상한: 87.50)
FairlyActiveMinutes: 55개 (하한: -33.00, 상한: 55.00)
LightlyActiveMinutes: 13개 (하한: -29.00, 상한: 459.00)
SedentaryMinutes: 4개 (하한: 31.00, 상한: 1871.00)
Calories: 11개 (하한: 434.50, 상한: 4294.50)


In [13]:
# 3-4. 이상치 제거
def remove_outliers_iqr(df, columns):
    """IQR 방법으로 이상치 제거"""
    df_clean = df.copy()
    
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # 이상치 제거
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    return df_clean

# 이상치 제거할 주요 컬럼
outlier_cols = ['TotalSteps', 'TotalDistance_km', 'TrackerDistance_km',
                'LoggedActivitiesDistance_km', 'VeryActiveDistance_km',
                'ModeratelyActiveDistance_km', 'LightActiveDistance_km',
                'VeryActiveMinutes','FairlyActiveMinutes', 
                'LightlyActiveMinutes', 
                'SedentaryMinutes', 'Calories']

print(f"이상치 제거 전: {len(actcp)}행")
actcp_final = remove_outliers_iqr(actcp, outlier_cols)
print(f"이상치 제거 후: {len(actcp_final)}행")
print(f"제거된 행: {len(actcp) - len(actcp_final)}행")
print("✓ 이상치 제거 완료")

이상치 제거 전: 1173행
이상치 제거 후: 857행
제거된 행: 316행
✓ 이상치 제거 완료


In [14]:
# 4-1. 최종 데이터 정보
print("=== 최종 데이터 정보 ===")
print(f"행 개수: {len(actcp_final)}")
print(f"열 개수: {len(actcp_final.columns)}")
print(f"\n데이터 타입:\n{actcp_final.dtypes}")

=== 최종 데이터 정보 ===
행 개수: 857
열 개수: 24

데이터 타입:
Id                                        str
ActivityDate                   datetime64[us]
TotalSteps                              int64
TotalDistance                         float64
TrackerDistance                       float64
LoggedActivitiesDistance              float64
VeryActiveDistance                    float64
ModeratelyActiveDistance              float64
LightActiveDistance                   float64
VeryActiveMinutes                       int64
FairlyActiveMinutes                     int64
LightlyActiveMinutes                    int64
SedentaryMinutes                        int64
Calories                                int64
TotalDistance_km                      float64
TrackerDistance_km                    float64
LoggedActivitiesDistance_km           float64
VeryActiveDistance_km                 float64
ModeratelyActiveDistance_km           float64
LightActiveDistance_km                float64
weekday                           

In [15]:
print("\n=== 주요 컬럼 기본 통계량 ===")
display(actcp_final[['TotalSteps', 'TotalDistance_km', 'Calories', 'VeryActiveMinutes', 'SedentaryMinutes']].describe())


=== 주요 컬럼 기본 통계량 ===


,TotalSteps,TotalDistance_km,Calories,VeryActiveMinutes,SedentaryMinutes
count,857.000000,857.000000,857.000000,857.000000,857.000000
mean,6866.263711,7.651501,2167.103851,10.536756,963.859977
std,3404.298302,3.798501,504.192488,15.049804,273.107693
min,1004.000000,1.046071,928.000000,0.000000,125.000000
25%,4131.000000,4.522245,1819.000000,0.000000,732.000000
50%,6633.000000,7.402964,2095.000000,1.000000,1018.000000
75%,9471.000000,10.508991,2525.000000,19.000000,1195.000000
max,16901.000000,18.346475,3439.000000,59.000000,1729.000000
